## Day 15: Beverage Bandits

https://adventofcode.com/2018/day/15

### Part 1

Blimey. I'll structure this in functions, passing state awkwardly, so I can check that each step is working correctly. Use persistent data structures so nothing goes wrong while doing so.

In [1]:
from collections import namedtuple
from pyrsistent import pset, pvector, pmap, pdeque
from itertools import count


State = namedtuple('State', 'cave units')
Unit = namedtuple('Unit', 'is_elf health attack')


def parse_cave(cave_data):
    cave = pmap()
    units = pmap()
    
    for row, line in enumerate(cave_data):
        for column, c in enumerate(line.rstrip()):
            if c != '#':
                cave = cave.set((row, column), pset())
                
                if c != '.':
                    units = units.set((row, column), Unit(c == 'E', 200, 3))
    
    # Link neighbouring squares
    for row, column in cave:
        for d in (-1, 1):
            if (row + d, column) in cave:
                cave = cave.set((row, column),
                                cave[(row, column)].add((row + d, column)))       
            if (row, column + d) in cave:
                cave = cave.set((row, column),
                                cave[(row, column)].add((row, column + d)))
                    
    return State(cave, units)


def state_string(state):
    cave, units = state
    
    output = ''
    
    rs, cs = zip(*cave)
    
    for row in range(min(rs) - 1, max(rs) + 2):
        units_in_row = []
        for col in range(min(cs) - 1, max(rs) + 2):
            pos = (row, col)
            
            if pos in cave:
                if pos in units:
                    output += 'E' if units[pos].is_elf else 'G'
                    units_in_row.append(units[pos])
                else:
                    output += '.'
            else:
                output += '#'
                
        if units_in_row:
            output += '   ' + ', '.join(f"{'E' if u.is_elf else 'G'}({u.health})"
                                        for u in units_in_row)
        output += '\n'
        
    return output
                
    
test_data = '''#######   
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#   
#######'''.splitlines()

test_state = parse_cave(test_data)
print(state_string(test_state))

#######
#.G...#   G(200)
#...EG#   E(200), G(200)
#.#.#G#   G(200)
#..G#E#   G(200), E(200)
#.....#
#######



Do a breadth-first search to find the closest square in range of an enemy. Instead of returning the first one found, for each level of search depth find all paths to squares in range. Order these paths by row-column endpoint then row-column first step, and select the first. The next step on the chosen path is returned.

In [2]:
def are_enemies(unit_1, unit_2):
    return unit_1.is_elf != unit_2.is_elf


def next_position(state, unit_position):
    
    # Find all the other unit positions
    other_positions = pset(unit for unit in state.units if unit != unit_position)
    
    # Find all the enemy positions
    enemy_positions = pvector(unit for unit in other_positions
                              if are_enemies(state.units[unit_position], state.units[unit]))
    
    # Find all neighbouring squares to enemies where there isn't another unit there
    squares_in_range = pset(set().union(*[state.cave[unit] 
                                          for unit in enemy_positions])) - other_positions
    
    # If already in range then stay still
    if unit_position in squares_in_range:
        return unit_position
    
    # Do a breadth first search until either a square in range is found or there
    # are no more paths
    
    shortest_path_to = pmap({unit_position: pvector([unit_position])})
    seen = pset({unit_position})
    check_from = pset({unit_position})
    
    while check_from:
        paths = pmap()
        
        for p in check_from:
            next_steps = state.cave[p] - seen - other_positions
            
            for n in next_steps:
                next_path = shortest_path_to[p].append(n)
                paths = paths.set(n, paths.get(n, pvector()).append(next_path))
                
        for p in paths:
            shortest_path_to = shortest_path_to.set(p, min(paths[p]))
        # Has a path to a square in range been found?
        paths_to_ranges = [shortest_path_to[p] for p in paths if p in squares_in_range]
        if paths_to_ranges:
            # All the current paths are the same length so tie-break by
            # destination then first step in reading order
            # Return the next step, the second element of the chosen path 
            return min(paths_to_ranges, key=lambda p: (p[-1], p[1]))[1]
                
        check_from = pset(paths)
        seen = seen | pset(paths)
        
    # No path to enemy found    
    return unit_position

In [3]:
for unit_position in test_state.units:
    print(unit_position, next_position(test_state, unit_position))

(4, 5) (4, 5)
(1, 2) (1, 3)
(2, 5) (2, 5)
(2, 4) (2, 4)
(4, 3) (3, 3)
(3, 5) (3, 5)


That matches the first round of the test.

In [4]:
def move_unit(state, old_position, new_position):
    return State(state.cave, 
                 state.units.discard(old_position).set(new_position, 
                                                       state.units[old_position]))


def attack(state, unit_position):
    enemy_positions = pset(unit for unit in state.units
                           if are_enemies(state.units[unit_position], state.units[unit]))
    enemies_in_range = state.cave[unit_position] & enemy_positions
    
    if enemies_in_range:
        victim = min(enemies_in_range, 
                     key=lambda e: (state.units[e].health, e))
        victim_unit = state.units[victim]
        victim_after_attack = Unit(victim_unit.is_elf, 
                                   victim_unit.health - state.units[unit_position].attack,
                                   victim_unit.attack)
        
        if victim_after_attack.health <= 0:
            state.units[victim]
            return State(state.cave, state.units.discard(victim))
        else:
            return State(state.cave, state.units.set(victim, victim_after_attack))
    else:
        return state

In [5]:
def turn(state):
    units = sorted(state.units)
    
    for unit_position in units:
        # Check unit wasn't destroyed in a previous attack
        if unit_position in state.units:
            new_position = next_position(state, unit_position)
            state = move_unit(state, unit_position, new_position)
            state = attack(state, new_position)
            
    return state

In [6]:
running_state = test_state

for i in range(1, 48):
    running_state = turn(running_state)
    if i in [1,2,23,24,25,26,27,28,47]:
        print(f'After {i} rounds:')
        print(state_string(running_state))

After 1 rounds:
#######
#..G..#   G(200)
#...EG#   E(197), G(197)
#.#G#G#   G(200), G(197)
#...#E#   E(197)
#.....#
#######

After 2 rounds:
#######
#...G.#   G(200)
#..GEG#   G(200), E(188), G(194)
#.#.#G#   G(194)
#...#E#   E(194)
#.....#
#######

After 23 rounds:
#######
#...G.#   G(200)
#..G.G#   G(200), G(131)
#.#.#G#   G(131)
#...#E#   E(131)
#.....#
#######

After 24 rounds:
#######
#..G..#   G(200)
#...G.#   G(131)
#.#G#G#   G(200), G(128)
#...#E#   E(128)
#.....#
#######

After 25 rounds:
#######
#.G...#   G(200)
#..G..#   G(131)
#.#.#G#   G(125)
#..G#E#   G(200), E(125)
#.....#
#######

After 26 rounds:
#######
#G....#   G(200)
#.G...#   G(131)
#.#.#G#   G(122)
#...#E#   E(122)
#..G..#   G(200)
#######

After 27 rounds:
#######
#G....#   G(200)
#.G...#   G(131)
#.#.#G#   G(119)
#...#E#   E(119)
#...G.#   G(200)
#######

After 28 rounds:
#######
#G....#   G(200)
#.G...#   G(131)
#.#.#G#   G(116)
#...#E#   E(113)
#....G#   G(200)
#######

After 47 rounds:
#######
#G....#   G(20

That seems to be working.

For the full battle we need to check if all units of either side are killed mid-round, so the turn function above is integrated into this ugly mess. I have no appetite for making this more elegant at this point.

In [7]:
def battle(state, debug_every=None):
    for i in count():
        if (not any(unit.is_elf for unit in state.units.values()) 
            or not any(not unit.is_elf for unit in state.units.values())):
            print(state_string(state))
            print(f'{i} * {sum(unit.health for unit in state.units.values())} = {i * sum(unit.health for unit in state.units.values())}')
            return i * sum(unit.health for unit in state.units.values())
        
        if debug_every and i % debug_every == 0:
            print(f'Round {i}')
            print(state_string(state))
            
        units = sorted(state.units)
    
        for unit_position in units:
            # Check unit wasn't destroyed in a previous attack
            if unit_position in state.units:
                if (not any(unit.is_elf for unit in state.units.values()) 
                    or not any(not unit.is_elf for unit in state.units.values())):
                    print(state_string(state))
                    print(f'{i} * {sum(unit.health for unit in state.units.values())} = {i * sum(unit.health for unit in state.units.values())}')
                    return i * sum(unit.health for unit in state.units.values())
                
                new_position = next_position(state, unit_position)
                state = move_unit(state, unit_position, new_position)
                state = attack(state, new_position)      

In [8]:
battle(test_state)

#######
#G....#   G(200)
#.G...#   G(131)
#.#.#G#   G(59)
#...#.#
#....G#   G(200)
#######

47 * 590 = 27730


27730

In [9]:
test_suite = [
    '''#######
#G..#E#
#E#E.E#
#G.##.#
#...#E#
#...E.#
#######
'''.splitlines(),
    '''#######
#E..EG#
#.#G.E#
#E.##E#
#G..#.#
#..E#.#
#######
'''.splitlines(),
    '''#######  
#E.G#.#
#.#G..#
#G.#.G#
#G..#.#
#...E.#
#######
'''.splitlines(),
    '''#######
#.E...#
#.#..G#
#.###.#
#E#G#G#
#...#G#
#######
'''.splitlines(),
    '''#########
#G......#
#.E.#...#
#..##..G#
#...##..#
#...#...#
#.G...G.#
#.....G.#
#########
'''.splitlines()
]

for test_case in test_suite:
    print(battle(parse_cave(test_case)))

#######
#...#E#   E(200)
#E#...#   E(197)
#.E##.#   E(185)
#E..#E#   E(200), E(200)
#.....#
#######

37 * 982 = 36334
36334
#######
#.E.E.#   E(164), E(197)
#.#E..#   E(200)
#E.##.#   E(98)
#.E.#.#   E(200)
#...#.#
#######

46 * 859 = 39514
39514
#######
#G.G#.#   G(200), G(98)
#.#G..#   G(200)
#..#..#
#...#G#   G(95)
#...G.#   G(200)
#######

35 * 793 = 27755
27755
#######
#.....#
#.#G..#   G(200)
#.###.#
#.#.#.#
#G.G#G#   G(98), G(38), G(200)
#######

54 * 536 = 28944
28944
#########
#.G.....#   G(137)
#G.G#...#   G(200), G(200)
#.G##...#   G(200)
#...##..#
#.G.#...#   G(200)
#.......#
#.......#
#########

20 * 937 = 18740
18740


The test cases pass.

In [10]:
state = parse_cave(open('input', 'r'))

%time battle(state)

################################
####.#######.....########.....##
##............#..#######.......#
#...#.........#######..#......##
########.......######..##.....##
########.........####..###....##
#...###.#.....##...##.....#...##
##....#..#....####..##........##
##..#....#..#######...........##
#####....G...#######..........##   G(101)
#########......####...###......#
#########......G......###.....##   G(104)
########......#####...##########
#########....#######..##########
#########...#########.##########
#########G..#########.##########   G(200)
######......#########.##########
#.###.......#########.##########
#.##.......G#########..#########   G(200)
#.........G..#######...#########   G(107)
#...#....G....#####....#########   G(200)
#####...G..G.G.....G.....#######   G(200), G(53), G(200), G(200)
####......G.G............#######   G(200), G(200)
####.......G............########   G(200)
#####.....G.G............#######   G(86), G(176)
####....#..G....#...#....#######   G(200)
####.

212787

But the answer for the problem input is wrong. Let's check the movement is working correctly.

In [11]:
move_test = parse_cave('''#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########
'''.splitlines())

In [12]:
battle(move_test, debug_every=1)

Round 0
#########
#G..G..G#   G(200), G(200), G(200)
#.......#
#.......#
#G..E..G#   G(200), E(200), G(200)
#.......#
#.......#
#G..G..G#   G(200), G(200), G(200)
#########

Round 1
#########
#.G...G.#   G(200), G(200)
#...G...#   G(197)
#...E..G#   E(200), G(200)
#.G.....#   G(200)
#.......#
#G..G..G#   G(200), G(200), G(200)
#.......#
#########

Round 2
#########
#..G.G..#   G(200), G(200)
#...G...#   G(194)
#.G.E.G.#   G(200), E(197), G(200)
#.......#
#G..G..G#   G(200), G(200), G(200)
#.......#
#.......#
#########

Round 3
#########
#.......#
#..GGG..#   G(200), G(191), G(200)
#..GEG..#   G(200), E(185), G(200)
#G..G...#   G(200), G(200)
#......G#   G(200)
#.......#
#.......#
#########

Round 4
#########
#.......#
#..GGG..#   G(200), G(188), G(200)
#..GEG..#   G(200), E(173), G(200)
#G..G...#   G(200), G(200)
#......G#   G(200)
#.......#
#.......#
#########

Round 5
#########
#.......#
#..GGG..#   G(200), G(185), G(200)
#..GEG..#   G(200), E(161), G(200)
#G..G...#   G(200), G(200)


27828

Yes, after some debugging. The answer's still wrong though.

[This reddit thread](https://www.reddit.com/r/adventofcode/comments/a6f100/day_15_details_easy_to_be_wrong_on/) specifies some common hiccups and gives a couple of edge cases.

In [13]:
edge_case = parse_cave('''####
##E#
#GG#
####'''.splitlines())

battle(edge_case)

####
##.#
#.G#   G(200)
####

66 * 200 = 13200


13200

That should be 67 rather than 66. A gnome dies mid-round but its location is still in the list of units to take a turn. Another gnome moves into the dead gnome's square and then has a second turn when the dead gnome's turn comes along. I'll need to modify the code to track which units have been killed in each round.

In [14]:
edge_case_2 = parse_cave('''#####
#GG##
#.###
#..E#
#.#G#
#.E##
#####'''.splitlines())

battle(edge_case_2)

#######
#..####
#G#####   G(197)
#...###
#.#.###
#..####
#######

71 * 197 = 13987


13987

The second edge case gives the right answer.

To track the units killed mid-round the `attack` function is brought into the main `battle` function. What a mess.

In [15]:
def battle(state, debug_every=None):
    for i in count():
        if (not any(unit.is_elf for unit in state.units.values()) 
            or not any(not unit.is_elf for unit in state.units.values())):
            print(state_string(state))
            print(f'{i} * {sum(unit.health for unit in state.units.values())} = {i * sum(unit.health for unit in state.units.values())}')
            return i * sum(unit.health for unit in state.units.values())
        
        if debug_every and i % debug_every == 0:
            print(f'Round {i}')
            print(state_string(state))
            
        units = sorted(state.units)
        killed = pset()
        
        for unit_position in units:
            # Check unit wasn't destroyed in a previous attack
            if not unit_position in killed:
                if (not any(unit.is_elf for unit in state.units.values()) 
                    or not any(not unit.is_elf for unit in state.units.values())):
                    print(state_string(state))
                    print(f'{i} * {sum(unit.health for unit in state.units.values())} = {i * sum(unit.health for unit in state.units.values())}')
                    return i * sum(unit.health for unit in state.units.values())
                
                new_position = next_position(state, unit_position)
                state = move_unit(state, unit_position, new_position)
                unit_position = new_position
                
                # Attack
                enemy_positions = pset(unit for unit in state.units
                                       if are_enemies(state.units[unit_position], state.units[unit]))
                enemies_in_range = state.cave[unit_position] & enemy_positions

                if enemies_in_range:
                    victim = min(enemies_in_range, 
                                 key=lambda e: (state.units[e].health, e))
                    victim_unit = state.units[victim]
                    victim_after_attack = Unit(victim_unit.is_elf, 
                                               victim_unit.health - state.units[unit_position].attack,
                                               victim_unit.attack)

                    if victim_after_attack.health <= 0:
                        state = State(state.cave, state.units.discard(victim))
                        killed = killed.add(victim)
                    else:
                        state = State(state.cave, state.units.set(victim, victim_after_attack))

In [16]:
edge_case = parse_cave('''####
##E#
#GG#
####'''.splitlines())

battle(edge_case)

####
##.#
#.G#   G(200)
####

67 * 200 = 13400


13400

This answer for the edge case is now correct.

In [17]:
%time battle(state)

################################
####.#######.....########.....##
##............#..#######.......#
#...#.........#######..#......##
########.......######..##.....##
########.........####..###....##
#...###.#.....##...##.....#...##
##....#..#....####..##........##
##..#....#..#######...........##
#####....G...#######..........##   G(101)
#########......####...###......#
#########......G......###.....##   G(104)
########......#####...##########
#########....#######..##########
#########...#########.##########
#########G..#########.##########   G(200)
######......#########.##########
#.###.......#########.##########
#.##.......G#########..#########   G(200)
#.........G..#######...#########   G(107)
#...#....G....#####....#########   G(200)
#####...G..G.G.....G.....#######   G(200), G(53), G(200), G(200)
####......G.G............#######   G(200), G(200)
####.......G............########   G(200)
#####.....G.G............#######   G(83), G(176)
####....#..G....#...#....#######   G(200)
####.

215168

Finally, the right answer. That took tens of hours, I only continued because of the sunk cost fallacy.